# Ear Training Proof of Concept
In this notebook, we will be creating a proof of concept for an ear training application. The application will play a random note and the user will have to guess the note. The application will then provide feedback on whether the user guessed correctly or not.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..") # Move execution to root dir

In [2]:
import json
import random
import re
import time
from pathlib import Path
from typing import Union

from IPython.display import clear_output
from pygame import mixer

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
RESOURCES_DIR = Path("resources")
AUDIO_DIR = RESOURCES_DIR / "sounds/piano"
NOTE_DATA_FILE = RESOURCES_DIR / "note_data.json"

## Get Note Data

In [4]:
NOTES = {
    0: "A0", 1: "A#0/Bb0", 2: "B0", 3: "C1", 4: "C#1/Db1", 5: "D1", 
    6: "D#1/Eb1", 7: "E1", 8: "F1", 9: "F#1/Gb1", 10: "G1", 11: "G#1/Ab1", 
    12: "A1", 13: "A#1/Bb1", 14: "B1", 15: "C2", 16: "C#2/Db2", 17: "D2", 
    18: "D#2/Eb2", 19: "E2", 20: "F2", 21: "F#2/Gb2", 22: "G2", 23: "G#2/Ab2", 
    24: "A2", 25: "A#2/Bb2", 26: "B2", 27: "C3", 28: "C#3/Db3", 29: "D3", 
    30: "D#3/Eb3", 31: "E3", 32: "F3", 33: "F#3/Gb3", 34: "G3", 35: "G#3/Ab3", 
    36: "A3", 37: "A#3/Bb3", 38: "B3", 39: "C4", 40: "C#4/Db4", 41: "D4", 
    42: "D#4/Eb4", 43: "E4", 44: "F4", 45: "F#4/Gb4", 46: "G4", 47: "G#4/Ab4", 
    48: "A4", 49: "A#4/Bb4", 50: "B4", 51: "C5", 52: "C#5/Db5", 53: "D5", 
    54: "D#5/Eb5", 55: "E5", 56: "F5", 57: "F#5/Gb5", 58: "G5", 59: "G#5/Ab5", 
    60: "A5", 61: "A#5/Bb5", 62: "B5", 63: "C6", 64: "C#6/Db6", 65: "D6", 
    66: "D#6/Eb6", 67: "E6", 68: "F6", 69: "F#6/Gb6", 70: "G6", 71: "G#6/Ab6", 
    72: "A6", 73: "A#6/Bb6", 74: "B6", 75: "C7", 76: "C#7/Db7", 77: "D7", 
    78: "D#7/Eb7", 79: "E7", 80: "F7", 81: "F#7/Gb7", 82: "G7", 83: "G#7/Ab7", 
    84: "A7", 85: "A#7/Bb7", 86: "B7", 87: "C8"
}

def get_note_files(audio_dir: Union[str, Path]) -> list[str]:
    note_files = []
    for root, dirs, files in os.walk(audio_dir):
        note_files.extend([filename for filename in files if filename.endswith(".wav")])
    return sorted(note_files)
    
def get_note_data(audio_dir: Union[str, Path], save_loc: Union[str, Path]) -> list[dict]:
    note_files = get_note_files(audio_dir)
    note_data = []
    for idx, note_file in enumerate(note_files):
        full_note_name = NOTES[idx]
        note_data.append(
            {
                "idx": idx,
                "full_note_name": full_note_name,
                "note_name": re.sub(r'[0-9]+', '', full_note_name),
                "file": note_file,
            }
        )
    with open(save_loc, "w") as f:
        json.dump(note_data, f, indent=2)
    return note_data

In [5]:
if not NOTE_DATA_FILE.exists():
    note_data = get_note_data(AUDIO_DIR, NOTE_DATA_FILE)
else:
    with open(NOTE_DATA_FILE) as f:
        note_data = json.load(f)

## Play notes at random

In [6]:
mixer.init()  # Initialize the mixer module.

RANGE = (15, 65)  # Range of notes from which to play
SELECTED_NOTES = ["A", "C", "E", "G"] # Notes to play
INCORRECT_SLEEP_LENGTH = 1  # Length of time to wait after picking the incorrect note
CORRECT_SLEEP_LENGTH = 3  # Length of time to wait after picking the correct note

def note_is_allowed(note_data: dict, selected_notes: list[str]) -> bool:
    return (
        note_data["note_name"] in selected_notes 
        and note_data["idx"] >= RANGE[0] 
        and note_data["idx"] <= RANGE[1]
    )

NOTES_TO_PLAY = [note for note in note_data if note_is_allowed(note, SELECTED_NOTES)]

In [24]:
def get_accuracy_str(n_correct: int, n_attempts: int):
    if n_attempts == 0:
        return f"{n_correct} / {n_attempts}"
    return f"{n_correct} / {n_attempts} - {round(100 * n_correct / n_attempts, 2)}"

def get_note_prediction(note_sound_player: mixer.Sound, possible_choices: list[str]):
    print(
        f"Possible note choices: {possible_choices}.\n\n"
        "Type the note you think it is, 'q' to quit, "
        "or anything else to replay the sound. Click enter when done."
    )
    while True:
        note_sound_player.play()
        note_choice = input().strip().upper()
        if note_choice == "Q":
            print("Quitting...")
            raise KeyboardInterrupt()
        if note_choice in possible_choices:
            return note_choice

def run_game():
    true_notes = []
    predicted_notes = []
    n_correct = 0
    n_attempts = 0
    last_note = None
    quit = False
    while True:
        note = random.choice(NOTES_TO_PLAY)
        while note == last_note:
            note = random.choice(NOTES_TO_PLAY)
            
        note_sound_player = mixer.Sound(AUDIO_DIR / note["file"])
        possible_choices = list(SELECTED_NOTES)
        while True:
            print(f"Results: {get_accuracy_str(n_correct, n_attempts)}\n")
            try:
                pred = get_note_prediction(note_sound_player, possible_choices)
            except KeyboardInterrupt:
                clear_output()
                quit = True
                break
                
            n_attempts += 1
            true_notes.append(note["note_name"])
            predicted_notes.append(pred)
            if pred != note["note_name"]:
                print("\n\nIncorrect note. Try again.")
                possible_choices.remove(pred)
                time.sleep(INCORRECT_SLEEP_LENGTH)
                clear_output()
            else:
                print(f'\n\nCorrect! Full note: {note["full_note_name"]}')
                n_correct += 1
                time.sleep(CORRECT_SLEEP_LENGTH)
                clear_output()
                break
        
        if quit:
            print(f"Final results this session: {get_accuracy_str(n_correct, n_attempts)}")
            break
        last_note = note
        

In [ ]:
run_game()

Results: 5 / 11 - 45.45

Possible note choices: ['A', 'C', 'E', 'G'].

Type the note you think it is, 'q' to quit, or anything else to replay the sound. Click enter when done.
